In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse as sparse
import numpy as np

In [2]:
%%time
final_sparse = sparse.load_npz(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/sparse_final_df.npz')
co_occ_sparse = sparse.load_npz(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/co_occurrence_sparse.npz')

CPU times: user 140 ms, sys: 28.1 ms, total: 168 ms
Wall time: 199 ms


In [10]:
df_pickle = pd.read_pickle('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipe_list/ingr_map.pkl')
ingredients_clean = df_pickle[['id','replaced','count','raw_ingr']]

In [3]:
%%time
row,col = final_sparse.nonzero()
data = pd.DataFrame.from_dict({'recipe': row, 'ingredient': col})

CPU times: user 37.2 ms, sys: 17.8 ms, total: 55 ms
Wall time: 74.1 ms


In [4]:
co_occ_df = pd.DataFrame(co_occ_sparse.todense())

In [5]:
%%time
from_sparse_df = final_sparse.todense()
final_df = pd.DataFrame(from_sparse_df)

CPU times: user 1.09 s, sys: 1.86 s, total: 2.95 s
Wall time: 3.58 s


In [ ]:
final_df

In [ ]:
co_occ_df

In [6]:
%%time
dot_sparse = final_sparse.dot(co_occ_sparse)

CPU times: user 34 s, sys: 7.21 s, total: 41.3 s
Wall time: 49.4 s


In [7]:
%%time
dot_arr = dot_sparse.todense()

CPU times: user 8.12 s, sys: 34.4 s, total: 42.5 s
Wall time: 1min 23s


In [8]:
%%time
dot_df = pd.DataFrame(dot_arr)

CPU times: user 3.18 ms, sys: 15.4 ms, total: 18.5 ms
Wall time: 34.2 ms


In [9]:
dot_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,2,10,4,3,0,4,9,1,0,1,...,0,0,53,1,5,25,144,0,3650,3
1,1,4,2,1,1,1,11,0,2,1,...,0,0,12,2,1,19,106,0,1711,2
2,1,3,1,2,0,5,4,1,0,2,...,0,0,61,2,4,22,162,0,4432,2
3,0,1,0,0,0,0,4,0,0,0,...,0,0,0,0,1,8,20,0,240,0
4,0,0,0,0,0,3,0,2,0,0,...,0,0,39,2,7,7,98,0,2079,0


In [28]:
idx = pd.np.nonzero(final_sparse)
dot_sparse[idx] = 0

<ipython-input-28-94638dfad31d>:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  idx = pd.np.nonzero(final_sparse)
/Users/giancarlotissot/.pyenv/versions/3.8.6/envs/ingredient_matching/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [45]:
sparse.save_npz(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/rec_matrix.npz',dot_sparse)

KeyboardInterrupt: 

In [29]:
%%time
rec_mat = dot_sparse.todense()

CPU times: user 8.25 s, sys: 36.2 s, total: 44.4 s
Wall time: 1min 53s


In [33]:
%%time
rec_df = pd.DataFrame(rec_mat)

CPU times: user 507 µs, sys: 11 µs, total: 518 µs
Wall time: 566 µs


In [ ]:
(rec_df)

In [34]:
rec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,2,10,4,3,0,4,9,1,0,1,...,0,0,53,1,5,25,144,0,3650,3
1,1,4,2,1,1,1,11,0,2,1,...,0,0,12,2,1,19,106,0,1711,2
2,1,3,1,2,0,5,4,1,0,2,...,0,0,61,2,4,22,162,0,4432,2
3,0,1,0,0,0,0,4,0,0,0,...,0,0,0,0,1,8,20,0,240,0
4,0,0,0,0,0,3,0,2,0,0,...,0,0,39,2,7,7,98,0,2079,0


In [40]:
rec_df.iloc[0].sort_values(ascending=False)

2499    40228
5010    34493
6906    31645
3203    27884
2683    25944
        ...  
1387        0
1384        0
3723        0
3738        0
5179        0
Name: 0, Length: 8023, dtype: int64

In [43]:
def get_id(ingredient_str):
    '''Transforms string input to pre-processed ID'''
    ingredient_id = ingredients_clean[ingredients_clean['raw_ingr'] ==ingredient_str]
    ingredient_id.reset_index(inplace=True)
    return ingredient_id.loc[0,'id']

def get_name(ingredient_id):
    '''Transforms ID back to pre-processed string'''
    ingredient_name = ingredients_clean[ingredients_clean['id'] == ingredient_id]
    ingredient_name.reset_index(inplace=True)
    return ingredient_name.loc[0,'replaced']

In [49]:
def find_match(ingredient_id,num_matches):
    '''Returns list of ingredient IDs and count of occurances'''
    recipes = final_df[final_df[ingredient_id]==1]
    total = rec_df.loc[recipes.index].sum()
    ids = total.sort_values(ascending=False)
    return list(ids.head(num_matches).index)
    

# def output_func(input_ingredient,num_matches=10):
#     '''Combines other functions into a workflow'''
#     num_matches += 1
#     result = get_id(input_ingredient)
#     ids = find_match(result,num_matches)
#     names = list_to_names(ids)
#     names.pop(0)
#     return names

In [50]:
# def find_match(id_input,num_matches):
#     # Returns list of ingredient IDs and count of occurances
#     numpy_arr = np.nonzero(from_sparse_df[:,id_input])
#     indices = (-final_sparse[numpy_arr[0],:].sum(axis=0).A1).argsort()[:num_matches]
#     return indices
    
def list_to_names(ingredient_id_list):
    # Takes in list of ingredient IDs and returns list of names
    list_ = []
    for id in ingredient_id_list:
        list_.append(get_name(id))
    return list_

def output_func(input_ingredient,num_matches=10):
    # Combines other functions into a workflow
    num_matches += 1
    
    if type(input_ingredient) != list:
        input_ingredient = [input_ingredient]
    id_input = []
    for ingredient in input_ingredient:
        id_input.append(get_id(ingredient))
    id_list = find_match(id_input,num_matches)
    names = list_to_names(id_list)

#     names.pop(0)
    return names

In [ ]:
%%time
output_func(['salt','lettuce'])

In [ ]:
%%time
# calculate user recommendation matrix
user_matrix = np.dot(x, co_matrix)
idx = pd.np.nonzero(x)
user_matrix[idx] = 0
df_user_recommend = pd.DataFrame(user_matrix, columns = df.columns, index = df.index)

In [ ]:
df_user_recommend

In [42]:
get_name(6906)

'sugar'

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix

In [ ]:
recipe_category = CategoricalDtype(sorted(data.recipe.unique()), ordered=True)
ingredient_category = CategoricalDtype(sorted(data.ingredient.unique()), ordered=True)

In [ ]:
row = data['recipe'].astype(recipe_category).cat.codes
col = data['ingredient'].astype(ingredient_category).cat.codes

In [ ]:
data['Count'] = 1

In [ ]:
sparse_matrix = csr_matrix((data['Count'], (row, col)),shape=(recipe_category.categories.size,ingredient_category.categories.size))

In [ ]:
sparse_df = pd.SparseDataFrame(sparse_matrix,index=recipe_category.categories,columns=ingredient_category.categories,default_fill_value=0)